# [WIP] Anomaly Detection concept with a CTD profile
Comparing traditional QC with hard thresholds versus Anomaly Detection

## Objective:

Illustrate some QC procedures using CoTeDe.

An example on how to QC a CTD profile

- Introduce dataset that will be used
- 

Suggest to check profile_CTD notebook first for the basics on how to use CoTeDe

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.plotting import figure
import numpy as np
from scipy import stats

import cotede
from cotede.qc import ProfileQC
from cotede import qctests, datasets

In [ ]:
output_notebook()

## Data
CoTeDe comes with a few datasets for demonstration.
Here we will use a CTD cast from the PIRATA hydrographic collection, i.e. measurements from the Tropical Atlantic Ocean.
If curious about this data, check [CoTeDe's documentation](https://cotede.readthedocs.io) for more details.

Let's start by loading this dataset.

In [ ]:
data = cotede.datasets.load_ctd()

print("There is a total of {} observed levels.\n".format(len(data["TEMP"])))
print("The variables are: ", list(data.keys()))

This CTD was equipped with backup sensors to provide more robustness. Measurements from the secondary sensor are identified by a 2 in the end of the name, such as "TEMP2" for the secondary temperature sensor. Let's focus on the primary sensors.

To visualize this profile we will use Bokeh so you can explore the data and results.
For instance, in the following plot zoom in to better see the profiles of temperature and salinity.

In [ ]:
p1 = figure(plot_width=420, plot_height=600)
p1.circle(
    data['TEMP'], -data['PRES'], size=8, line_color="seagreen",
    fill_color="mediumseagreen", fill_alpha=0.3)
p1.xaxis.axis_label = "Temperature [C]"
p1.yaxis.axis_label = "Depth [m]"

p2 = figure(plot_width=420, plot_height=600)
p2.y_range = p1.y_range
p2.circle(
    data['PSAL'], -data['PRES'], size=8, line_color="seagreen",
    fill_color="mediumseagreen", fill_alpha=0.3)
p2.xaxis.axis_label = "Salinity"
p2.yaxis.axis_label = "Depth [m]"

p = row(p1, p2)
show(p)

Considering the unusual magnitudes and variability near the bottom, there are clearly bad measurements on this profile.
Let's start with a traditional QC and then we'll include the Anomaly Detection.

## Traditional QC with CoTeDe framework
*NOTE: If you are not familiar with CoTeDe, it might be helpfull to check first the notebook profile_CTD and come back after that.*

Let's start with the procedure recommended by the EuroGOOS for non-realtime data, which includes the climatology test comparison with the World Ocean Atlas (WOA).
If interested, check CoTeDe's manual for more details, including the reference, on the EuroGOOS recomendations.

In [ ]:
pqc = cotede.ProfileQC(data, cfg="eurogoos")

That's it, the temperature and salinity from the primary and secondary sensors were all evaluated.

Which criteria were flagged for the primary sensors?

In [ ]:
print("Flags for temperature:\n {}\n".format(list(pqc.flags["TEMP"].keys())))

print("Flags for salinity:\n {}\n".format(list(pqc.flags["PSAL"].keys())))

The flags are on IOC standard, thus 1 means good while 4 means bad.
0 is used when no QC test was applied.
For instance, the spike test is defined so that it depends on the previous and following measurements, thus the first and last data point of the array will always have a spike flag equal to 0.

How could we use that?
Let's check which are the unfeasible measurements of salinity, i.e. flagged as bad (flag=4) or probably bad (flag=3) according to the Global Range check.

In [ ]:
idx = pqc.flags["PSAL"]["global_range"] >= 3
pqc["PSAL"][idx]

The flag "overall" combines all criteria, and it is the maximum flag value among all the criteria applied, as recommended by the IOC.
Therefore, if one measurement is flagged bad (flag=4) in a single test, it will get a flag 4.
Likewise, a measurement with flag 1 means that the maximum value from all applied tests was 1, hence there is no suggestion of being a bad measurement.

In [ ]:
pqc.flags["PSAL"]["overall"]

#### EuroGOOS automatic QC
Let's visualize what the automatic EuroGOOS procedure can detect for temperature and salinity. The concept is the same for all variables evaluated, i.e. there is a flag "overall" for "TEMP" and another one for "PSAL".

In [ ]:
# ToDo: Include a shaded area for unfeasible values

idx_good = pqc.flags["TEMP"]["overall"] <= 2
idx_bad = pqc.flags["TEMP"]["overall"] >= 3

p1 = figure(plot_width=420, plot_height=600, title="QC according to EuroGOOS")
p1.circle(data['TEMP'][idx_good], -data['PRES'][idx_good], size=8, line_color="seagreen", fill_color="mediumseagreen", fill_alpha=0.3, legend_label="Good values")
p1.triangle(data['TEMP'][idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3, legend_label="Bad values")
p1.xaxis.axis_label = "Temperature [C]"
p1.yaxis.axis_label = "Depth [m]"
p1.legend.location = "top_right"

idx_good = pqc.flags["PSAL"]["overall"] <= 2
idx_bad = pqc.flags["PSAL"]["overall"] >= 3

p2 = figure(plot_width=420, plot_height=600, title="QC according to EuroGOOS")
p2.y_range = p1.y_range
p2.circle(data['PSAL'][idx_good], -data['PRES'][idx_good], size=8, line_color="seagreen", fill_color="mediumseagreen", fill_alpha=0.3, legend_label="Good values")
p2.triangle(data['PSAL'][idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3, legend_label="Bad values")
p2.xaxis.axis_label = "Pratical Salinity"
p2.yaxis.axis_label = "Depth [m]"
p2.legend.location = "top_right"

p = row(p1, p2)
show(p)

The result from the EuroGOOS recommendations is pretty good and it is one of my favorite QC setup when considering only the traditional methods.
Most of the bad measurements were automatically detected, but if you zoom in below 800m you will notice some questionable measurements that were not flagged.

In the following section we will see why did that happened and how can we improve that.

## Limitations of the traditional unidimensional test
The traditional approach to QC oceanographic data is based on projecting the data in a new dimension and then apply hard thresholds, such as the spike test (see the notebook profile_CTD).
To avoid false positives, i.e. flag good data as bad, those thresholds are usually tolerant enought to accept extreme events.
For instance, if we define a gradient threshold too tight we risk to flag the intense gradients in the thermocline as bad by mistake.

In the following figure you have the salinity in the left and the respective "spikeness".
With the slider you can choose what is the threshold, such that the measurements above that value would be flagged as bad (red triangle).
Note that with a threshold of 0.05, we would flag the some measurements near the surface but that wouldn't be enough to flag that jump in 826m depth.
For reference, the EuroGOOS's recommended threshold for deep ocean is 0.3

The same issue is observed in the temperature of this profile.

In [ ]:
from bokeh.models import ColumnDataSource, CustomJS, Slider

threshold = Slider(title="threshold", value=0.05, start=0.0, end=6.0, step=0.05, orientation="horizontal")


tmp = dict(
    depth=-pqc["PRES"],
    temp=pqc["PSAL"],
    temp_good=pqc["PSAL"].copy(),
    temp_bad=pqc["PSAL"].copy(),
    spike=np.absolute(pqc.features["PSAL"]["spike"]),
    spike_good=np.absolute(pqc.features["PSAL"]["spike"]),
    spike_bad=np.absolute(pqc.features["PSAL"]["spike"]),    
)
idx = tmp["spike"] > threshold.value
tmp["temp_good"][idx] = np.nan
tmp["temp_bad"][~idx] = np.nan
tmp["spike_good"][idx] = np.nan
tmp["spike_bad"][~idx] = np.nan


source = ColumnDataSource(data=tmp)


callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    var temp = data['temp']
    var temp_good = data['temp_good']
    var temp_bad = data['temp_bad']
    var spike = data['spike']
    var spike_good = data['spike_good']
    var spike_bad = data['spike_bad']
    for (var i = 0; i < temp.length; i++) {
        if (spike[i] > f) {
            temp_good[i] = "NaN"
            temp_bad[i] = temp[i]
            spike_good[i] = "NaN"
            spike_bad[i] = spike[i]
        } else {
            temp_good[i] = temp[i]
            temp_bad[i] = "NaN"
            spike_good[i] = spike[i]
            spike_bad[i] = "NaN"
        }
    }
    source.change.emit();
""")


threshold.js_on_change('value', callback)


p1 = figure(plot_width=420, plot_height=600)
p1.circle("temp_good", "depth", source=source, size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p1.triangle("temp_bad", "depth", source=source, size=8, line_color="red", fill_color="red", fill_alpha=0.3)

p2 = figure(plot_width=420, plot_height=600)
p2.y_range = p1.y_range
p2.circle("spike_good", "depth", source=source, size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p2.triangle("spike_bad", "depth", source=source, size=8, line_color="red", fill_color="red", fill_alpha=0.3)
# inputs = row(threshold)
#threshold = column(slider)


p = column(threshold, row(p1, p2))
show(p)

Because the thresholds were wisely defined with tolerant values, the traditional QC procedure does a great job flagging bad values, i.e. there is a high confidence that a measurement flagged as bad is indeed a bad one.
To avoid the mistake of flagging good measurements as bad ones, To achieve that, some bad measurements are wrongly flagged as good.

That is achieved by a lower confidenceis confident in detecting bad measurements, but tend to mistake a few bad measurements as good ones.


It would be nice if could somehow account by how much a 

In [ ]:
print("PRES: {}".format(pqc["PRES"][825]))
print("TEMP: {}".format(pqc["TEMP"][825]))
for c in ["gradient", "spike", "woa_normbias"]:
    print("{}: {}".format(c, pqc.features["TEMP"][c][825]))

EuroGOOS
- Gradient below 500m: 3.0
- Spike below 500m: 2.0
- Climatology: 6 standard deviations

None of the criteria failed individually.

For the climatology comparison we have a scaled value in standard deviations, but how large was the estimated spike? How uncommon was that? Could we combine the information?

In [ ]:
pqc.flags["PSAL"]

Let's look at the salinity in respect to the spike and WOA normalized bias.
Near the bottom of the profile there some bad salinity measurement, which are mostly identified with the spike test.
A few measurements aren't critically bad in respect to the spike or the climatology individually.
One of the goals of the Anomaly Detection is to combine multiple features to an overall decision, so that

In [ ]:

idx_good = pqc.flags["PSAL"]["spike_depthconditional"] <= 2
idx_bad = pqc.flags["PSAL"]["spike_depthconditional"] >= 3

p1 = figure(plot_width=500, plot_height=600)
p1.circle(pqc.features["PSAL"]["spike"][idx_good], -pqc['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p1.triangle(pqc.features["PSAL"]["spike"][idx_bad], -pqc['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)

p2 = figure(plot_width=500, plot_height=600)
p2.y_range = p1.y_range
p2.circle(pqc['PSAL'][idx_good], -pqc['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p2.line(pqc.features["PSAL"]["woa_mean"] - 6 * pqc.features["PSAL"]["woa_std"], -data['PRES'], line_width=4, line_color="orange", alpha=0.4)
p2.line(pqc.features["PSAL"]["woa_mean"] + 6 * pqc.features["PSAL"]["woa_std"], -data['PRES'], line_width=4, line_color="orange", alpha=0.4)
p2.triangle(data['PSAL'][idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)

p = row(p1, p2)
show(p)

In [ ]:
pqc = cotede.ProfileQC(data, cfg="cotede")

print(pqc.features["TEMP"].keys())
pqc.features["TEMP"]["anomaly_detection"][824:827]

In [ ]:
t_spike = pqc.features["TEMP"]["anomaly_detection"]

idx_good = np.absolute(t_spike) <= 2
idx_bad = np.absolute(t_spike) > 2

p1 = figure(plot_width=420, plot_height=500)
p1.circle(data['TEMP'][idx_good], -data['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p1.triangle(data['TEMP'][idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)
p1.xaxis.axis_label = "Temperature [C]"
p1.yaxis.axis_label = "Depth [m]"

p2 = figure(plot_width=420, plot_height=500)
p2.y_range = p1.y_range
p2.circle(t_spike[idx_good], -data['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p2.triangle(t_spike[idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)
p2.xaxis.axis_label = "Spike(T)"
p2.yaxis.axis_label = "Depth [m]"


s_spike = pqc.features["PSAL"]["woa_normbias"]

idx_good = np.absolute(s_spike) <= 2
idx_bad = np.absolute(s_spike) > 2

p3 = figure(plot_width=420, plot_height=500)
p3.y_range = p1.y_range
p3.circle(data['PSAL'][idx_good], -data['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p3.triangle(data['PSAL'][idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)
p3.xaxis.axis_label = "Salinity"
p3.yaxis.axis_label = "Depth [m]"

p4 = figure(plot_width=420, plot_height=500)
p4.y_range = p1.y_range
p4.circle(s_spike[idx_good], -data['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p4.triangle(s_spike[idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)
p4.xaxis.axis_label = "Spike(S)"
p4.yaxis.axis_label = "Depth [m]"

p = column(row(p1, p2), row(p3, p4))
show(p)

In [ ]:
pqc.features["TEMP"]["anomaly_detection"]

In [ ]:
y_spike

In [ ]:
N = y_spike.size
n_greater = np.array([y_spike[np.absolute(y_spike) >= t].size/N for t in np.absolute(y_spike)])

p = figure(plot_width=840, plot_height=400)
p.circle(np.absolute(y_spike), n_greater, size=8, line_color="green", fill_color="green", fill_alpha=0.3)
show(p)

In [ ]:
from scipy.stats import exponweib

spike_scale = np.arange(0.0005, 0.2, 1e-3)
param = [1.078231, 0.512053, 0.0004, 0.002574]
tmp = exponweib.sf(spike_scale, *param[:-2], loc=param[-2], scale=param[-1])
p = figure(plot_width=840, plot_height=400)
# p.line(x_ref, pdf, line_color="orange", line_width=8, alpha=0.7, legend_label="PDF")
p.circle(spike_scale, tmp, size=8, line_color="green", fill_color="green", fill_alpha=0.3)
show(p)

In [ ]:
N = y_spike.size
n_greater = np.array([y_spike[np.absolute(y_spike) >= t].size/N for t in np.absolute(y_spike)])


p1 = figure(plot_width=420, plot_height=600)
p1.circle(data['TEMP'][idx_good], -data['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p1.triangle(data['TEMP'][idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)

p2 = figure(plot_width=420, plot_height=600)
p2.circle(SF[idx_good], -data['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p2.triangle(SF[idx_bad], -data['PRES'][idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)
p = row(p1, p2)
show(p)

In [ ]:
p1 = figure(plot_width=420, plot_height=600)
p1.circle(data['TEMP'][idx_good], -data['PRES'][idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)

In [ ]:
pqc_eurogoos = cotede.ProfileQC(data, cfg="eurogoos")
flag_eurogoos = pqc_eurogoos.flags["TEMP"]["overall"]

pqc = cotede.ProfileQC(data, cfg="cotede")
pqc.features["TEMP"].keys()

In [ ]:

AD_good = pqc.features["TEMP"]["anomaly_detection"][flag_eurogoos <= 2]
AD_bad = pqc.features["TEMP"]["anomaly_detection"][flag_eurogoos >= 3]


In [ ]:
min(AD_good)

In [ ]:
    x = AD_bad
    x = AD_good
    bins = 100
    hist, edges = np.histogram(x, density=True, bins=bins)

    #title = 'test'
    # p = figure(title=title, tools='', background_fill_color="#fafafa")
    p = figure(plot_width=750, plot_height=300,
        background_fill_color="#fafafa")
        # tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    # p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="PDF")
    # p.line(x, cdf, line_color="orange", line_width=2, alpha=0.7, legend_label="CDF")

    p.y_range.start = 0
    # p.legend.location = "center_right"
    # p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    show(p)

In [ ]:
def draw_histogram(x, bins=50):
    """Plot an histogram
    
    Create an histogram from the output of numpy.hist().
    We will create several histograms in this notebook so let's save this as a function to
    reuse this code.
    """
    x = x[np.isfinite(x)]
    hist, edges = np.histogram(x, density=True, bins=bins)

    #title = 'test'
    # p = figure(title=title, tools='', background_fill_color="#fafafa")
    p = figure(plot_width=750, plot_height=300,
        background_fill_color="#fafafa")
        # tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    # p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="PDF")
    # p.line(x, cdf, line_color="orange", line_width=2, alpha=0.7, legend_label="CDF")

    p.y_range.start = 0
    # p.legend.location = "center_right"
    # p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

In [ ]:
p = draw_histogram(y_spike[idx_good], bins=50)
show(p)

In [ ]:
data.attrs

In [ ]:
import oceansdb
WOADB = oceansdb.WOA()

woa = WOADB['TEMP'].extract(var=['mean', 'standard_deviation'], doy=data.attrs['datetime'], lat=data.attrs['LATITUDE'], lon=data.attrs['LONGITUDE'], depth=data['PRES'])

In [ ]:
pqc.features["TEMP"]["woa_mean"] - 6 * pqc.features["TEMP"]["woa_std"]

In [ ]:
pqc = cotede.ProfileQC(data)

In [ ]:
pqc.flags['TEMP'].keys()

In [ ]:
pqc.features['TEMP']


In [ ]:
p = figure(plot_width=500, plot_height=600)
p.circle(y_spike, -data['PRES'], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
show(p)

In [ ]:
idx_valid

In [ ]:
np.percentile(y_tukey53H[np.isfinite(y_tukey53H)], 25)

In [ ]:
idx = y_tukey53H[np.absolute(y_tukey53H) < 6]

p = draw_histogram(y_tukey53H[idx & idx_valid])
show(p)

In [ ]:
idx = idx_valid & np.isfinite(y_tukey53H)

mu_estimated, sigma_estimated = stats.norm.fit(y_tukey53H[idx])

print("Estimated mean: {:.3f}, and standard deviation: {:.3f}".format(mu_estimated, sigma_estimated))

In [ ]:
y_tukey53H_scaled = (y_tukey53H - mu_estimated) / sigma_estimated

In [ ]:
p = figure(plot_width=500, plot_height=600)
p.circle(y_tukey53H_scaled, -data['PRES'], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
show(p)

In [ ]:
idx_good = pqc.flags["PSAL"]["overall"] <= 2
idx_bad = pqc.flags["PSAL"]["overall"] >= 3

pressure = -pqc["PRES"]
salinity = pqc["PSAL"]
woa_normbias = pqc.features["PSAL"]["woa_normbias"]


p1 = figure(plot_width=420, plot_height=500)
p1.circle(salinity[idx_good], pressure[idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p1.triangle(salinity[idx_bad], pressure[idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)
p1.xaxis.axis_label = "Salinity"
p1.yaxis.axis_label = "Depth [m]"

p2 = figure(plot_width=420, plot_height=500)
p2.y_range = p1.y_range
p2.circle(woa_normbias[idx_good], pressure[idx_good], size=8, line_color="green", fill_color="green", fill_alpha=0.3)
p2.triangle(woa_normbias[idx_bad], pressure[idx_bad], size=8, line_color="red", fill_color="red", fill_alpha=0.3)
p2.xaxis.axis_label = "WOA normalized bias"
p2.yaxis.axis_label = "Depth [m]"

p = row(p1, p2)
show(p)